# Import Necessary Libraries

In [91]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
# import gensim
# from gensim import corpora

#Visualization
from spacy import displacy
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])
from nltk.tokenize import word_tokenize

#Data loading/ Data manipulation
import pandas as pd
import numpy as np

# Ensure required NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Import json library
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anadu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anadu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anadu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anadu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import the CSV file

In [92]:
file_path = 'data/Resume Dataset/Resume/Resume.csv'
data = pd.read_csv(file_path)

# Display the first few rows
# print(data.head())

data


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [93]:
# put data into a df
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

# Export the data to another CSV file
# df.to_csv('data/Resume Dataset/Resume/Resume_clean.csv', index=False)


         ID                                         Resume_str   
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...  \
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


# Clean the data

In [94]:
# Remove the Resume_html column
df.drop('Resume_html', axis=1, inplace=True)

# Remove the ID column
df.drop('ID', axis=1, inplace=True)

# Display the first few rows
# print(df.head())

In [95]:
print(df.shape)

# Find distinct categories in Category column
print(df['Category'].value_counts())


# Display the first few rows
# print(df.head())

# Reset the index
df = df.reset_index(drop=True)

# See the shape of the data
print(df.shape)




(2484, 2)
Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64
(2484, 2)


In [96]:
# Remove commas from the 'Resume_str' column
df['Resume_str'] = df['Resume_str'].str.replace('\n', ' ', regex=True)
df['Resume_str'] = df['Resume_str'].str.replace('\r', ' ', regex=True)
df['Resume_str'] = df['Resume_str'].str.strip()

df['Resume_str'] = df['Resume_str'].str.replace('"', '', regex=True)
df['Resume_str'] = df['Resume_str'].str.replace(',', ' ', regex=True)

df['Resume_str'] = df['Resume_str'].fillna('').astype(str)

# Remove rows where all values are NaN
df = df.dropna(how='all')

# Optionally remove rows where a specific column (e.g., 'Resume_str') is empty
df = df[df['Resume_str'].str.strip().astype(bool)]

# Display the first few rows
# print(df.head())

# Display size
print(df.shape)


(2483, 2)


In [97]:
# Export the data to another CSV file
df.to_csv('data/Resume Dataset/Resume/Resume_Clean.csv', index=False)

In [98]:
# !python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "skill_patterns_lowercase.jsonl"


# write a script to turn all the jsonl text lowercase
# They are in this pattern {"label": "HARD SKILL", "pattern": [{"TEXT": "Programming"}]}
# with open(skill_pattern_path, "r") as infile, open(output_file, "w") as outfile:
#     for line in infile:
#         # Load each line as a JSON object
#         skill_entry = json.loads(line)
        
#         # Check if the pattern exists and modify the TEXT field
#         if "pattern" in skill_entry:
#             for item in skill_entry["pattern"]:
#                 if "TEXT" in item:
#                     item["TEXT"] = item["TEXT"].lower()  # Convert to lowercase
        
#         # Write the updated JSON object back to the output file
#         json.dump(skill_entry, outfile)
#         outfile.write("\n")



In [99]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

We will create two python functions to extract all the skills within a resume and create an array containing all the skills. Later we are going to apply this function to our dataset and create a new feature called skill. This will help us visualize trends and patterns within the dataset

In [100]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset

def get_hard_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "HARD SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset

def unique_skills(x):
    return list(set(x))

# Function to extract roles in all caps
def extract_roles(text):
    # Find the portion of the text up to the first lowercase letter
    match = re.match(r'^([A-Z\s]+)[^A-Z]', text)
    if match:
        # Extract the matched string, remove the last letter and space, then lowercase
        role = match.group(1).strip()[:-1].lower()
        return role
    return ""

# Cleaning Resume Text using nltk

In [101]:
# Go through each row of the 'Resume_str' column and clean the text
df['Resume_str'] = df['Resume_str'].apply(lambda x: re.sub(r'\s+', ' ', x))

# Remove special characters and digits
df['Resume_str'] = df['Resume_str'].apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['Resume_str'] = df['Resume_str'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Show the first few rows
print(df.head())




                                          Resume_str Category
0  HR ADMINISTRATOR MARKETING ASSOCIATE HR ADMINI...       HR
1  HR SPECIALIST US HR OPERATIONS Summary Versati...       HR
2  HR DIRECTOR Summary years experience recruitin...       HR
3  HR SPECIALIST Summary Dedicated Driven Dynamic...       HR
4  HR MANAGER Skill Highlights HR SKILLS HR Depar...       HR


In [102]:
# Create a CSV for the Processed Data
df.to_csv('data/Resume Dataset/Resume/Resume_Processed.csv', index=False)



In [103]:
processed = df

processed["skills"] = processed["Resume_str"].str.lower().apply(get_skills)
processed["skills"] = processed["skills"].apply(unique_skills)

# Display the first few rows
print(processed.head())

                                          Resume_str Category   
0  HR ADMINISTRATOR MARKETING ASSOCIATE HR ADMINI...       HR  \
1  HR SPECIALIST US HR OPERATIONS Summary Versati...       HR   
2  HR DIRECTOR Summary years experience recruitin...       HR   
3  HR SPECIALIST Summary Dedicated Driven Dynamic...       HR   
4  HR MANAGER Skill Highlights HR SKILLS HR Depar...       HR   

                                              skills  
0  [accounting, integrity, customer service, inte...  
1  [support, training, process improvement, commu...  
2  [support, training, information management, cu...  
3  [training, customer service, certificate, conf...  
4  [support, training, problem solving, integrity...  


In [104]:
processed["hard_skills"] = processed["Resume_str"].str.lower().apply(get_hard_skills)
processed["hard_skills"] = processed["hard_skills"].apply(unique_skills)

# Display the first few rows
print(processed.head())

                                          Resume_str Category   
0  HR ADMINISTRATOR MARKETING ASSOCIATE HR ADMINI...       HR  \
1  HR SPECIALIST US HR OPERATIONS Summary Versati...       HR   
2  HR DIRECTOR Summary years experience recruitin...       HR   
3  HR SPECIALIST Summary Dedicated Driven Dynamic...       HR   
4  HR MANAGER Skill Highlights HR SKILLS HR Depar...       HR   

                                              skills   
0  [accounting, integrity, customer service, inte...  \
1  [support, training, process improvement, commu...   
2  [support, training, information management, cu...   
3  [training, customer service, certificate, conf...   
4  [support, training, problem solving, integrity...   

                                         hard_skills  
0  [certification, programming, development, stat...  
1  [development, web, communication, develop, rep...  
2  [databases, retention, development, hiring, te...  
3  [dynamic, documentation, database, communicati...

In [105]:
processed['roles'] = processed['Resume_str'].apply(extract_roles)

# Display the first few rows
print(processed.head())

                                          Resume_str Category   
0  HR ADMINISTRATOR MARKETING ASSOCIATE HR ADMINI...       HR  \
1  HR SPECIALIST US HR OPERATIONS Summary Versati...       HR   
2  HR DIRECTOR Summary years experience recruitin...       HR   
3  HR SPECIALIST Summary Dedicated Driven Dynamic...       HR   
4  HR MANAGER Skill Highlights HR SKILLS HR Depar...       HR   

                                              skills   
0  [accounting, integrity, customer service, inte...  \
1  [support, training, process improvement, commu...   
2  [support, training, information management, cu...   
3  [training, customer service, certificate, conf...   
4  [support, training, problem solving, integrity...   

                                         hard_skills   
0  [certification, programming, development, stat...  \
1  [development, web, communication, develop, rep...   
2  [databases, retention, development, hiring, te...   
3  [dynamic, documentation, database, communicat

In [81]:


# Find all distinct roles and extract them to a jsonl file with the pattern format
# roles = processed['roles'].unique()
# roles = [role for role in roles if role]
# roles = [{"label": "ROLE", "pattern": role} for role in roles]
# with open("role_patterns.jsonl", "w") as f:
#     for role in roles:
#         f.write(json.dumps(role) + "\n")



In [106]:
# Save the data to a new CSV file
processed.to_csv("data/Resume Dataset/Resume/Resume_With_Skills_And_Roles.csv", index=False)

In [107]:
sent = nlp(
    """

    """
)
displacy.render(sent, style="ent", jupyter=True)

c:\Python310\lib\site-packages\spacy\displacy\__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [108]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])


# options=[{"ents": "Job-Category", "colors": "#ff3232"},{"ents": "SKILL", "colors": "#56c426"}]
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
displacy.render(sent, style="ent", jupyter=True, options=options)